# Project - Stock Information AI Assistant


In [ ]:
!pip install finnhub-python

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import finnhub
from typing import Dict, List, Any, Optional
from datetime import datetime

In [ ]:
import logging

# Configure root logger
logging.basicConfig(
    level=logging.INFO,              # Set level: DEBUG, INFO, WARNING, ERROR
    format="%(asctime)s [%(levelname)s] %(message)s", 
    force=True                       # Ensures reconfiguration if you rerun this cell
)

logger = logging.getLogger(__name__)  # Use a global logger object
logger.info("Logger initialized!")

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")

if openai_api_key:
    logger.info(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    logger.error("OpenAI API Key not set")

if FINNHUB_API_KEY:
    logger.info(f"FINNHUB_API_KEY exists!")
else:
    logger.error("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini" # not using gpt-5-mini as openai doesn't let you stream responses till you are a verified organisation :(
openai = OpenAI()
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

In [ ]:
system_message = f"""
You are "TickerBot" — a concise, factual, educational assistant specializing in U.S. stocks.  
Your job: quickly and accurately explain stock and company information in plain English. NEVER give investment advice, buy/sell recommendations, or price predictions.

## PRIVACY ABOUT IMPLEMENTATION
- Do not reveal any internal implementation details to users. Never display or mention internal tool names, API names, developer notes, configured flags, date-range limits, or other system/developer constraints in user-facing replies.
- All runtime/tool constraints and capability detection are internal. Present only user-facing capabilities in plain language.

## USER-FACING CAPABILITIES
- When asked "What can you do?", list only stock-relevant actions in plain language. Example reply:
  "I can look up tickers, show the latest quotes, provide key company financials and latest earnings details, summarize recent company or market headlines, and give a brief market overview."
- Do not list internal utilities or developer tools as user-facing capabilities.

## GENERAL PRINCIPLES
- Answer only what was asked for. 
- Be brief, clear, and professional while still maintaining a warm tone. Use short paragraphs and one-line bullet explanations when requested.
- Return only what the system provides; do not invent, infer, or extrapolate unavailable data.
- Never offer or advertise any feature the environment does not actually support. Avoid offering attachments, direct downloads, or full-text article retrieval unless the system explicitly provides those outputs.

## Behavior Rules
- Stay professional and neutral at all times.  
- Clarify only when user intent is ambiguous; never guess.  
- Only disclose information the user explicitly requested.  
- Never explain system limits (e.g., API ranges, date limits) ever.  
- Summaries should be tight and relevant, not verbose.  

## NEWS & HEADLINES
- When interpreting date-related or temporal reasoning requests (e.g., “latest earnings,” “recent news,” “Q1 results”) Call `get_current_time` to determine the current date.
- Present news/headlines in concise bullet lines when requested. Default recent-window behavior is internal; do not describe or expose internal default windows or limits to the user.
- If the system only returns headlines/summaries, present those and do not offer to fetch full-text or additional ranges unless the user explicitly asks and the environment supports that action.

## FOLLOW-UP & CLARIFYING QUESTIONS
- If no matching stock symbol is found, ask the user to clarify the name or ticker. Mention you only support U.S. stocks. If they confirm the symbol but no data exists, state that no results were found.
- Never append unsolicited menus, multi-choice lists, or repeated "Would you like..." prompts at the end of a normal reply.
- Ask a single direct clarifying question only when strictly necessary to fulfill the user's request (for example: ambiguous company name or missing ticker). That single question must be the final line of the reply.
- If the user's intent is clear, proceed and return results. Do not request confirmations or offer options unless required to complete the task.

## MISSING-DATA / NOTE RULES
- Do NOT call out missing/unavailable single fields unless:
  1) the missing field was explicitly requested by the user; OR
  2) multiple (>1) key metrics required to answer the user's request are unavailable and their absence materially prevents a useful answer.
- If condition (1) or (2) applies, include at most one concise "Note:" line naming the specific missing field(s) and nothing else.
- Otherwise omit any comment about single, non-central missing fields.

## ERROR HANDLING
- If a company/ticker cannot be found: "I couldn't find that company on U.S. exchanges. Please provide the ticker or a clearer company name."
- If partial results are returned: present available data and at most one short note: "Some data wasn't available: <field1>, <field2>."
- If you do not know the answer, say plainly: "I don't have that information."

## ADVICE / DISCLAIMER (CONDITIONAL)
- Do NOT include an investment-advice disclaimer in normal greetings or factual replies.
- Show a single-line disclaimer only when the user explicitly asks for recommendations/advice or uses clear advice-seeking language (e.g., "should I buy", "recommend", "what should I do", "financial advice").
- Disclaimer text when triggered: "I can't provide investment advice or buy/sell recommendations. I can provide facts and explain metrics."

EXAMPLES (templates — follow exactly)

1) Clear factual request (no unsolicited follow-up)
User: "What's AAPL last trade?"
Assistant:
Last trade timestamp: 2025-08-28 20:00:00 UTC (U.S. market close — 4:00 PM ET)

(—end: no menu, no offers—)

2) Ambiguous request — single clarifying question only
User: "Show me Apple"
Assistant:
I found two matches. Do you mean Apple Inc. (AAPL) or Apple Hospitality REIT (APLE)?

(—end: single question final line—)

3) News returned as headlines only (do not offer unsupported fetches)
User: "Recent NVDA news"
Assistant:
- 2025-08-28: Headline A — short summary.
- 2025-08-26: Headline B — short summary.
(—end: do not offer full-article fetch or extended-range offers unless explicitly requested and supported—)
"""


In [ ]:
def get_current_time() -> Dict[str, Any]:
    """
    Retrieve the current UTC time in ISO format with timezone.
    Returns a dictionary for consistency with other tools.
    """
    try:
        current_time = datetime.utcnow().isoformat() + 'Z'
        return {
            "success": True,
            "current_time": current_time
        }
    except Exception as e:
        return {"success": False, "error": f"Failed to get time: {str(e)[:100]}"}

In [ ]:
get_current_time_function = {
    "name": "get_current_time",
    "description": "Get the current UTC time in ISO format (YYYY-MM-DDTHH:MM:SS.ssssssZ). Useful for temporal reasoning, date calculations, or setting time ranges for queries like news.",
    "parameters": {
        "type": "object",
        "properties": {},  # No parameters needed
        "required": []
    }
}
get_current_time_tool = {"type": "function", "function": get_current_time_function}

In [ ]:
def validate_symbol(symbol: str) -> bool:
    """Validate stock symbol format"""
    if not symbol or not isinstance(symbol, str):
        return False
    return symbol.isalnum() and 1 <= len(symbol) <= 5 and symbol.isupper()

def search_symbol(query: str) -> Dict[str, Any]:
    """Search for stock symbol using Finnhub client"""
    logger.info(f"Tool search_symbol called for {query}")
    try:
        if not query or len(query.strip()) < 1:
            return {"success": False, "error": "Invalid search query"}
        
        query = query.strip()[:50]
        result = finnhub_client.symbol_lookup(query)
        logger.info(f"Tool search_symbol {result}")
        
        if result.get("result") and len(result["result"]) > 0:
            first_result = result["result"][0]
            symbol = first_result.get("symbol", "").upper()
            
            if validate_symbol(symbol):
                return {
                    "success": True,
                    "symbol": symbol
                }
            else:
                return {"success": False, "error": "Invalid symbol format found"}
        else:
            return {"success": False, "error": "No matching US stocks found"}
            
    except Exception as e:
        return {"success": False, "error": f"Symbol search failed: {str(e)[:100]}"}

In [ ]:
search_symbol_function = {
    "name": "search_symbol",
    "description": "Search for a stock symbol / ticker symbol based on company name or partial name",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Company name or partial name to search for, extract only relevant name part and pass it here, keep this to less than 50 characters"
            }
        },
        "required": [
            "query"
        ]
    }
}

search_symbol_tool = {"type": "function", "function": search_symbol_function}

In [ ]:
def _format_big_number_from_millions(value_millions: Any) -> str:
    """
    Finnhub returns some large metrics (marketCapitalization, enterpriseValue, revenueTTM)
    in MILLIONS USD. Convert to full USD and format with M/B/T suffixes.
    """
    if value_millions is None:
        return "Unavailable"
    try:
        value = float(value_millions) * 1_000_000  # convert millions -> full USD
    except (TypeError, ValueError):
        return "Unavailable"

    trillion = 1_000_000_000_000
    billion = 1_000_000_000
    million = 1_000_000

    if value >= trillion:
        return f"{value / trillion:.2f}T USD"
    if value >= billion:
        return f"{value / billion:.2f}B USD"
    if value >= million:
        return f"{value / million:.2f}M USD"
    return f"{value:.2f} USD"


def _safe_metric(metrics: Dict[str, Any], key: str) -> Any:
    """
    Return metric value if present; otherwise "Unavailable".
    We intentionally return the raw value for numeric metrics (no rounding/format)
    except for the specially formatted big-number fields handled elsewhere.
    """
    if metrics is None:
        return "Unavailable"
    val = metrics.get(key)
    return val if val is not None else "Unavailable"


def get_company_financials(symbol: str) -> Dict[str, Any]:
    """
    Fetch and return a curated set of 'basic' financial metrics for `symbol`.
    - Calls finnhub_client.company_basic_financials(symbol, 'all')
    - Formats market cap, enterprise value, revenue (Finnhub returns these in millions)
    - Returns success flag and readable keys
    """
    logger.info(f"Tool get_company_financials called for {symbol}")
    try:
        if not symbol or not symbol.strip():
            return {"success": False, "error": "Invalid stock symbol"}

        symbol = symbol.strip().upper()

        # --- API Call ---
        financials_resp = finnhub_client.company_basic_financials(symbol, "all")

        # Finnhub places primary values under "metric"
        metrics = financials_resp.get("metric", {})
        if not metrics:
            return {"success": False, "error": "No financial metrics found"}

        # --- Build result using helpers ---
        result = {
            "success": True,
            "symbol": symbol,
            "financials": {
                "Market Cap": _format_big_number_from_millions(metrics.get("marketCapitalization")),
                "Enterprise Value": _format_big_number_from_millions(metrics.get("enterpriseValue")),
                "P/E Ratio (TTM)": _safe_metric(metrics, "peBasicExclExtraTTM"),
                "Forward P/E": _safe_metric(metrics, "forwardPE"),
                "Gross Margin (TTM)": _safe_metric(metrics, "grossMarginTTM"),
                "Net Profit Margin (TTM)": _safe_metric(metrics, "netProfitMarginTTM"),
                "EPS (TTM)": _safe_metric(metrics, "epsTTM"),
                "EPS Growth (5Y)": _safe_metric(metrics, "epsGrowth5Y"),
                "Dividend Yield (Indicated Annual)": _safe_metric(metrics, "dividendYieldIndicatedAnnual"),
                "Current Ratio (Quarterly)": _safe_metric(metrics, "currentRatioQuarterly"),
                "Debt/Equity (Long Term, Quarterly)": _safe_metric(metrics, "longTermDebt/equityQuarterly"),
                "Beta": _safe_metric(metrics, "beta"),
                "52-Week High": _safe_metric(metrics, "52WeekHigh"),
                "52-Week Low": _safe_metric(metrics, "52WeekLow"),
            }
        }

        return result

    except Exception as e:
        # keep error message short but useful for debugging
        return {"success": False, "error": f"Failed to fetch metrics: {str(e)[:200]}"}

In [ ]:
get_company_financials_function = {
    "name": "get_company_financials",
    "description": "Fetch and return a curated set of basic financial metrics for a stock symbol. Calls Finnhub's company_basic_financials API, formats large numbers (market cap, enterprise value, revenue) in M/B/T USD, and shows metrics like P/E ratios, EPS, margins, dividend yield, debt/equity, beta, and 52-week range. Returns 'Unavailable' for missing values.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol to fetch metrics for. Example: 'AAPL' for Apple Inc."
            }
        },
        "required": [
            "symbol"
        ]
    }
}


get_company_financials_tool = {"type": "function", "function": get_company_financials_function}

In [ ]:
def get_stock_quote(symbol: str) -> dict:
    """
    Fetch the latest stock quote for a given ticker symbol using Finnhub's /quote endpoint.
    Returns current price, daily high/low, open, previous close, percent change, and readable timestamp.
    """
    logger.info(f"Tool get_stock_quote called for {symbol}")
    try:
        if not symbol or len(symbol.strip()) < 1:
            return {"success": False, "error": "Invalid symbol provided"}
        
        symbol = symbol.strip().upper()
        data = finnhub_client.quote(symbol)

        if not data or "c" not in data:
            return {"success": False, "error": "No quote data found"}
        
        # Convert epoch timestamp to ISO UTC if present
        timestamp = data.get("t")
        if timestamp and isinstance(timestamp, (int, float)):
            timestamp = datetime.utcfromtimestamp(timestamp).isoformat() + "Z"
        else:
            timestamp = "Unavailable"
        
        return {
            "success": True,
            "symbol": symbol,
            "current_price": round(data.get("c", 0), 2) if data.get("c") is not None else "Unavailable",
            "change": round(data.get("d", 0), 2) if data.get("d") is not None else "Unavailable",
            "percent_change": f"{round(data.get('dp', 0), 2)}%" if data.get("dp") is not None else "Unavailable",
            "high_price": round(data.get("h", 0), 2) if data.get("h") is not None else "Unavailable",
            "low_price": round(data.get("l", 0), 2) if data.get("l") is not None else "Unavailable",
            "open_price": round(data.get("o", 0), 2) if data.get("o") is not None else "Unavailable",
            "previous_close": round(data.get("pc", 0), 2) if data.get("pc") is not None else "Unavailable",
            "timestamp": timestamp
        }
    except Exception as e:
        return {"success": False, "error": f"Quote retrieval failed: {str(e)[:100]}"}

In [ ]:
get_stock_quote_function = {
  "name": "get_stock_quote",
  "description": "Retrieve the latest stock quote for a given symbol, including current price, daily high/low, open, previous close, and percent change. Data is near real-time. Avoid constant polling; use websockets for streaming updates.",
  "parameters": {
    "type": "object",
    "properties": {
      "symbol": {
        "type": "string",
        "description": "Stock ticker symbol to fetch the latest quote for. Example: 'AAPL', 'MSFT'."
      }
    },
    "required": ["symbol"]
  }
}

get_stock_quote_tool = {"type": "function", "function": get_stock_quote_function}


In [ ]:
def get_company_news(symbol: str, _from: str, to: str):
    """
    Fetch the top latest company news for a stock symbol within a date range.
    - Ensures the range does not exceed ~1 months (35 days).
    - Best practice: Keep searches to a month or less to avoid too much data.

    Args:
        symbol (str): Stock ticker (e.g., "AAPL").
        _from (str): Start date in YYYY-MM-DD format.
        to (str): End date in YYYY-MM-DD format.

    Returns:
        list or dict: Cleaned news data or error message.
    """
    # Validate date format
    logger.info(f"Tool get_company_news called for {symbol} from {_from} to {to}")
    try:
        start_date = datetime.strptime(_from, "%Y-%m-%d")
        end_date = datetime.strptime(to, "%Y-%m-%d")
    except ValueError:
        return {"success": False, "error": "Invalid date format. Use YYYY-MM-DD."}

    # Check date range
    delta_days = (end_date - start_date).days
    if delta_days > 35:
        return {
            "success": False, 
            "error": f"Date range too large ({delta_days} days). "
                     "Please use a range of 1 months or less."
        }

    # Fetch data
    try:
        news = finnhub_client.company_news(symbol, _from=_from, to=to)
    except Exception as e:
        return {"success": False, "error": str(e)}

    # Do not want to report just the latest news in the time period
    if len(news) <= 10:
    # If 10 or fewer articles, take all
        selected_news = news
    else:
        # Take first 5 (oldest) and last 5 (newest)
        selected_news = news[:5] + news[-5:]

    # Clean & transform objects
    cleaned_news = []
    for article in selected_news:
        cleaned_news.append({
            "summary": article.get("summary"),
            "source": article.get("source"),
            "published_at": datetime.utcfromtimestamp(article["datetime"]).strftime("%Y-%m-%d %H:%M:%S UTC"),
            "related": article.get("related")
        })

    return {"success": True, "news": cleaned_news}

In [ ]:
get_company_news_function = {
    "name": "get_company_news",
    "description": "Fetch the top most recent company news articles for a given stock symbol. ⚠️ Avoid querying more than a 1-month range at a time as it may return too much data. Only tells news about company within last 1 year. An error is returned if the requested time range exceeds 1 month.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol, e.g., 'AAPL'."
            },
            "_from": {
                "type": "string",
                "description": "Start date in YYYY-MM-DD format. Ensure it is not more than 1 year ago from today. Ensure it is before or equal to the date in to."
            },
            "to": {
                "type": "string",
                "description": "End date in YYYY-MM-DD format. Ensure it is not more than 1 year ago. Ensure it is after or equal to the date in from."
            }
        },
        "required": [
            "symbol",
            "_from",
            "to"
        ]
    }
}

get_company_news_tool = {"type": "function", "function": get_company_news_function}

In [ ]:
def get_market_news(category: str = "general"):
    """
    Fetch the latest market news for a given category.

    Args:
        category (str): News category. One of ["general", "forex", "crypto", "merger"].

    Returns:
        list or dict: A cleaned list of news articles or error message.
    """
    logger.info(f"Tool get_market_news called for category '{category}'")

    try:
        news = finnhub_client.general_news(category)
    except Exception as e:
        logger.error(f"Tool get_market_news API call failed!")
        return {"success": False, "error": str(e)}

    # Do not want to report just the latest news in the time period
    if len(news) <= 10:
    # If 10 or fewer articles, take all
        selected_news = news
    else:
        # Take first 5 (oldest) and last 5 (newest)
        selected_news = news[:5] + news[-5:]

    # Clean & transform objects
    cleaned_news = []
    for article in selected_news:
        cleaned_news.append({
            "headline": article.get("headline"),
            "summary": article.get("summary"),
            "source": article.get("source"),
            "category": article.get("category"),
            "related": article.get("related")
        })

    return {"success": True, "news": cleaned_news}

In [ ]:
get_market_news_function = {
  "name": "get_market_news",
  "description": "Fetch the latest market news by category. Returns the top 10 news articles with headline, summary, source, category, published time (UTC), and URLs. Categories: general, forex, crypto, merger. Use this to quickly get relevant financial news.",
  "parameters": {
    "type": "object",
    "properties": {
      "category": {
        "type": "string",
        "description": "News category to fetch. One of: general, forex, crypto, merger."
      }
    },
    "required": ["category"]
  }
}

get_market_news_tool = {"type": "function", "function": get_market_news_function}

In [ ]:
def get_earnings_calendar(symbol: str = "", _from: str = "", to: str = ""):
    """
    Fetch LATEST earnings calendar data for a stock symbol within a date range.
    - End date must be within the last month. (Free tier only allows last 1 month data)
    - Shows historical and upcoming earnings releases with EPS and revenue data.
    Args:
        symbol (str): Stock ticker (e.g., "AAPL"). Leave empty for all companies.
        _from (str): Start date in YYYY-MM-DD format.
        to (str): End date in YYYY-MM-DD format.
    Returns:
        list or dict: Cleaned earnings calendar data or error message.
    """
    logger.info(f"Tool get_earnings_calendar called for {symbol or 'all symbols'} from {_from} to {to}")
    
    # Validate date format if provided
    if _from or to:
        try:
            start_date = datetime.strptime(_from, "%Y-%m-%d") if _from else None
            end_date = datetime.strptime(to, "%Y-%m-%d") if to else None
            
            # Check date range if both dates provided
            # Check if end_date is within 1 month (≈30 days) of today
            if end_date:
                today = datetime.utcnow()
                if (today - end_date).days > 30:
                    return {
                        "success": False,
                        "error": "End date must be within the last month."
                    }
        except ValueError:
            return {"success": False, "error": "Invalid date format. Use YYYY-MM-DD."}
    
    # Fetch earnings calendar data
    try:
        earnings_data = finnhub_client.earnings_calendar(_from=_from, to=to, symbol=symbol, international=False)
    except Exception as e:
        logger.error(f"Error fetching earnings calendar: {e}")
        return {"success": False, "error": str(e)}
    
    # Check if data exists
    if not earnings_data or "earningsCalendar" not in earnings_data:
        return {"success": False, "error": "No earnings data available for the specified criteria."}
    
    earnings_list = earnings_data["earningsCalendar"]
    
    if not earnings_list:
        return {"success": True, "earnings": [], "message": "No earnings releases found for the specified period."}
    
    # Clean & transform earnings data
    cleaned_earnings = []
    for earning in earnings_list:
        # Format hour description
        hour_map = {
            "bmo": "Before Market Open",
            "amc": "After Market Close", 
            "dmh": "During Market Hours"
        }
        
        cleaned_earnings.append({
            "symbol": earning.get("symbol"),
            "date": earning.get("date"),
            "quarter": f"Q{earning.get('quarter')} {earning.get('year')}",
            "announcement_time": hour_map.get(earning.get("hour", ""), earning.get("hour", "Unknown")),
            "eps_actual": earning.get("epsActual"),
            "eps_estimate": earning.get("epsEstimate"),
            "revenue_actual": earning.get("revenueActual"),
            "revenue_estimate": earning.get("revenueEstimate")
        })
    
    return {"success": True, "earnings": cleaned_earnings}

In [ ]:
get_earnings_calendar_function = {
    "name": "get_earnings_calendar",
    "description": "Fetch latest earnings calendar showing historical and upcoming earnings releases for companies. Shows EPS and revenue estimates vs actuals. End date must be within the last month.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol, e.g., 'AAPL'. Leave empty to get earnings for all companies in the date range."
            },
            "_from": {
                "type": "string", 
                "description": "Start date in YYYY-MM-DD format. Ensure it is not more than 1 year ago from today. Ensure it is before or equal to the date in to."
            },
            "to": {
                "type": "string",
                "description": "End date in YYYY-MM-DD format. Ensure it is not more than 1 year ago. Ensure it is after or equal to the date in from. To date must be within the last month."
            }
        },
        "required": [
            "_from",
            "to"
        ]
    }
}

get_earnings_calendar_tool = {"type": "function", "function": get_earnings_calendar_function}

In [ ]:
# List of tools:
tools = [search_symbol_tool, get_company_financials_tool, get_stock_quote_tool, get_company_news_tool, get_market_news_tool, get_current_time_tool, get_earnings_calendar_tool]
tool_functions = {
    "search_symbol": search_symbol,
    "get_company_financials": get_company_financials,
    "get_stock_quote": get_stock_quote,
    "get_company_news": get_company_news,
    "get_market_news": get_market_news,
    "get_earnings_calendar": get_earnings_calendar,
    "get_current_time": get_current_time
}

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def execute_tool_call(tool_call):
    func_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    logger.info(f"Executing tool: {func_name} with args: {args}")

    func = tool_functions.get(func_name)
    if not func:
        result = {"error": f"Function '{func_name}' not found"}
    else:
        try:
            result = func(**args)
        except Exception as e:
            logger.exception(f"Error executing {func_name}")
            result = {"error": str(e)}

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    }

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # Skip the first system message
    to_log = messages[1:]

    # Print each dict on its own line
    logger.info("\nMessages:\n" + "\n".join(str(m) for m in to_log) + "\n")

    while True:
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            tools=tools,
            stream=True
        )
        
        content = ""
        tool_calls = []
        finish_reason = None
        
        # Process the stream
        for chunk in response:
            choice = chunk.choices[0]
            finish_reason = choice.finish_reason
            
            # Stream content
            if choice.delta.content:
                content += choice.delta.content
                yield content
            
            # Collect tool calls
            if choice.delta.tool_calls:
                for tc_delta in choice.delta.tool_calls:
                    # Extend tool_calls list if needed
                    while len(tool_calls) <= tc_delta.index:
                        tool_calls.append({
                            "id": "",
                            "function": {"name": "", "arguments": ""}
                        })
                    
                    tc = tool_calls[tc_delta.index]
                    if tc_delta.id:
                        tc["id"] = tc_delta.id
                    if tc_delta.function:
                        if tc_delta.function.name:
                            tc["function"]["name"] = tc_delta.function.name
                        if tc_delta.function.arguments:
                            tc["function"]["arguments"] += tc_delta.function.arguments
        
        # If no tool calls, we're done
        if finish_reason != "tool_calls":
            return content
        
        # Execute tools
        ai_message = {
            "role": "assistant", 
            "content": content,
            "tool_calls": [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": tc["function"]
                } for tc in tool_calls
            ]
        }
        
        tool_responses = []
        for tool_call in ai_message["tool_calls"]:
            # Convert dict back to object for your existing function
            class ToolCall:
                def __init__(self, tc_dict):
                    self.id = tc_dict["id"]
                    self.function = type('obj', (object,), tc_dict["function"])
            
            tool_responses.append(execute_tool_call(ToolCall(tool_call)))
        
        messages.append(ai_message)
        messages.extend(tool_responses)

In [ ]:
gr.ChatInterface(fn=chat, type="messages", title="TickerBot", description="Ask about stock prices, company financials and market news!").launch(share=True)